In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import unicodedata

# cities = ['Berlin', 'Hamburg', 'Frankfurt','Munich','Stuttgart','Cologne','Hannover']
cities = ['Berlin', 'Hamburg', 'Frankfurt','Munich','Stuttgart','Cologne','Hannover']

def City_info(soup):
    
    ret_dict = {}
    ret_dict['city'] = soup.h1.get_text()
    ret_dict['country'] = soup.select(".infobox-data")[0].get_text()
    

    
    if soup.select_one('.mergedtoprow:-soup-contains("Population")') != None:
        l = soup.select_one('.mergedtoprow:-soup-contains("Population")')
        c_pop = l.findNext('td').get_text()
        ret_dict['population'] = c_pop
    
    if soup.select_one('.latitude') != None:
        o = soup.select_one('.latitude')
        ret_dict['latitude'] = o.get_text()

    if soup.select_one('.longitude') != None:    
        p = soup.select_one('.longitude')
        ret_dict['longitude'] = p.get_text()
    
    
    return ret_dict

list_of_city_info = []
for city in cities:
    url = 'https://en.wikipedia.org/wiki/{}'.format(city)
    web = requests.get(url,'html.parser')
    soup = bs(web.content)
    list_of_city_info.append(City_info(soup))
df_cities = pd.DataFrame(list_of_city_info)
# fixing latitude
df_cities['latitude'] = df_cities['latitude'].str[:-1].str.replace('°', '.', regex=False).str.replace('′', '', regex=False).str.replace('″', '', regex=False)
# fixing longitude
df_cities['longitude'] = df_cities['longitude'].str[:-1].str.replace('°', '.', regex=False).str.replace('′', '', regex=False).str.replace('″', '', regex=False)
# fixing population
df_cities['population'] = df_cities['population'].str.replace(',', '', regex=False)
# change population to number 
df_cities['population'] = pd.to_numeric(df_cities['population'])
# change latitude to number 
df_cities['latitude'] = pd.to_numeric(df_cities['latitude']).round(4)
df_cities['longitude'] = pd.to_numeric(df_cities['longitude']).round(4)
df_cities

,city,country,population,latitude,longitude
0,Berlin,Germany,3677472,52.3112,13.2418
1,Hamburg,Germany,1906411,53.3300,10.0000
2,Frankfurt,Germany,759224,50.0638,8.4056
3,Munich,Germany,1487708,48.0815,11.3430
4,Stuttgart,Germany,626275,48.4639,9.1048
5,Cologne,Germany,1073096,50.5611,6.5710
6,Hanover,Germany,535932,52.2200,9.4300
